In [6]:
from geopy.geocoders import Nominatim
import pandas as pd

In [7]:
from joblib import load

In [8]:
class parameters:
    Model: str = "XGBoost"
    House_Direction = "đông"
    Balcony_Direction = "tây"
    Toilets = 5
    Bedrooms= 5
    Floors= 5
    Legits= "đỏ"
    Facade= 5
    Entrance= 2.5
    Ward = "Tương Mai"
    District = "Hoàng Mai"
    Area = 20
    
def get_inputs(features):
    inputs = {
            "House Direction": features.House_Direction,
            "Balcony Direction": features.Balcony_Direction,
            "Toilets":int(features.Toilets),
            "Bedrooms":int(features.Bedrooms),
            "Legits":features.Legits,
            "Floors": int(features.Floors),
            "Facade":float(features.Facade),
            "Entrance":float(features.Entrance),
            "Area": float(features.Area)
        }
    inputs["Toilets"] = "10+" if inputs["Toilets"] > 10 else str(inputs["Toilets"])
    inputs["Bedrooms"] = "10+" if inputs["Bedrooms"] > 10 else str(inputs["Bedrooms"])
    inputs["Floors"] = "10+" if inputs["Floors"] > 10 else str(inputs["Floors"])
    
    legits = ""
    if "đỏ" in inputs["Legits"]:
        legits +="+đỏ"
    if "hồng" in inputs["Legits"]:
        legits += "+hồng"
    inputs["Legits"] = legits if len(legits) > 0 else "None"
    loc = Nominatim(user_agent="GetLoc")
    getLoc = loc.geocode(features.Ward+" "+features.District)
    if getLoc is None:
        raise HTTPException(status_code=400, 
                            detail = "Invalid Address")
    inputs["X"] = getLoc[-1][0]
    inputs["Y"] = getLoc[-1][1]
    data = pd.DataFrame({i:[inputs[i]] for i in inputs})
    data = data[['House Direction', 'Balcony Direction', 'Toilets','Bedrooms', 
           'Legits', 'Floors', 'Facade', 'Entrance', "Area",'X', 'Y'
            ]]
    cates = ["House Direction", "Balcony Direction",  "Toilets", "Legits", "Floors", "Bedrooms"]
    for f in cates:
         data[f] = data[f].astype("category")
            
    scaler = load("models/scaler1.pkl")
    features_ = ['Facade', 'Entrance', "Area"]
    data[features_] = scaler.transform(data[features_])
    
    scaler = load("models/scaler2.pkl")
    features_ = ['X', 'Y']
    data[features_] = scaler.transform(data[features_])
    return data

In [9]:
inputs = get_inputs(parameters())

In [10]:
def predict_price(features: parameters):
    inputs = get_inputs(features)
    results = {}
    xgb = load("models/best_tree.pkl")
    results["Price"] = str(xgb.predict(inputs)[0])
    return results

In [11]:
predict_price(parameters())

{'Price': '55.509457'}

In [12]:
xgb = load("models/best_tree.pkl")
xgb.predict(inputs)[0]

55.509457

In [ ]:
{
  "Model": "XGBoost",
  "House_Direction": "đông",
  "Balcony_Direction": "tây",
  "Toilets": 5,
  "Bedrooms": 5,
  "Floors": 4,
  "Legits": "đỏ",
  "Facade": 5,
  "Entrance": 5,
  "Area": 30,
  "Ward": "tương mai",
  "District": "hoàng mai"
}